In [1]:
import numpy as np
import pandas as pd

print(f'numpy=={np.__version__}')
print(f'pandas=={pd.__version__}')

numpy==1.17.4
pandas==0.25.3


## Series & Index

Series 는 파이썬의 리스트나 column vector 같은 1차원의 데이터 집합입니다.

In [2]:
data = [1, 3, 'str', None, np.nan, 5]
series = pd.Series(data)
print(f'length={len(series)}')
print(series)

length=6
0       1
1       3
2     str
3    None
4     NaN
5       5
dtype: object


Series 의 각 element 에 이름을 붙일 수 있습니다. 이를 인덱스라 합니다.

In [3]:
series2 = pd.Series(data, index=list('abcdef'))
series2

a       1
b       3
c     str
d    None
e     NaN
f       5
dtype: object

Series 에는 직접 명명한 인덱스의 값, 혹은 위치 인덱스로 접근할 수 있습니다. 또한 namedtuple 처럼 인덱스의 값을 접근할 수 있습니다.

In [4]:
print(series[0])    # 1
print(series2[1])   # 3
print(series2['c']) # str
print(series2.c)    # nan

1
3
str
str


인덱스가 중복되면 해당 값들이 모두 출력됩니다. 그러나 이때는 위치 인덱스로 접근이 되지 않습니다. 일반적으로 인덱스는 고유값 (unique key) 가 될 수 있도록 하는 것이 좋습니다.

In [5]:
series2 = pd.Series(data, index=list('abcdee'))
series2['e']
# series2[0]

e    NaN
e      5
dtype: object

`pandas.data_range` 는 DateTime 형식의 series 를 생성하는 함수입니다. `freq` 를 조절하여 등간격의 날짜/시간 열을 만들 수 있습니다.

In [6]:
print(pd.date_range('20191217', periods=3, freq='D'))
print(pd.date_range('20191217', periods=3, freq='5D'))
print(pd.date_range('20191217', periods=3, freq='H'))

DatetimeIndex(['2019-12-17', '2019-12-18', '2019-12-19'], dtype='datetime64[ns]', freq='D')
DatetimeIndex(['2019-12-17', '2019-12-22', '2019-12-27'], dtype='datetime64[ns]', freq='5D')
DatetimeIndex(['2019-12-17 00:00:00', '2019-12-17 01:00:00',
               '2019-12-17 02:00:00'],
              dtype='datetime64[ns]', freq='H')


In [7]:
index = pd.date_range('20191217', periods=3, freq='D')
series = pd.Series([10, 20, 30], index=index)
series

2019-12-17    10
2019-12-18    20
2019-12-19    30
Freq: D, dtype: int64

In [8]:
series2 = pd.Series([40, 50, 60], index=index[::-1])
series2

2019-12-19    40
2019-12-18    50
2019-12-17    60
Freq: -1D, dtype: int64

Index 는 각 값의 식별자 역할을 합니다. 두 개의 series 를 더할 경우에 같은 index 별로 묶어서 그 값이 더해집니다.

In [9]:
series + series2

2019-12-17    70
2019-12-18    70
2019-12-19    70
Freq: D, dtype: int64

Index 를 따로 설정하지않으면 0 부터 1씩 증가하는 index 를 자동으로 만듭니다.

In [10]:
pd.Series([10, 20, 30]) + pd.Series([40, 50, 60])

0    50
1    70
2    90
dtype: int64

두 Series 중 하나의 인덱스가 존재하지 않으면 덧셈은 수행되지 않습니다.

In [11]:
series = pd.Series([10, 20, 30], index=index)
series2 = pd.Series([40, 50], index=index[:2])
series + series2

2019-12-17    50.0
2019-12-18    70.0
2019-12-19     NaN
Freq: D, dtype: float64

## DataFrame

DataFrame 은 2 차원 형식의 tabular 데이터입니다. 만드는 방식은 다양합니다. 각 column 별로 {column namne: row} 로 표현된 dict 를 입력할 수도 있습니다. 길이가 없는 값은 모든 row 에 반복하여 동일하게 입력됩니다.

In [12]:
data = {
    'A' : 1., 
    'B' : pd.Timestamp('20130102'),
    'C' : pd.Series(1,index=list(range(4)),dtype='float32'),
    'D' : np.array([3] * 4,dtype='int32'),
    'E' : pd.Categorical(["test","train","test","train"]),
    'F' : 'foo' }
df = pd.DataFrame(data)

print(df)

     A          B    C  D      E    F
0  1.0 2013-01-02  1.0  3   test  foo
1  1.0 2013-01-02  1.0  3  train  foo
2  1.0 2013-01-02  1.0  3   test  foo
3  1.0 2013-01-02  1.0  3  train  foo


DataFrame.head(), DataFrame.tail() 를 통하여 데이터의 형식과 예시를 빠르게 확인할 수 있습니다.

In [13]:
df.head(2)

,A,B,C,D,E,F
0,1.0,2013-01-02,1.0,3,test,foo
1,1.0,2013-01-02,1.0,3,train,foo


In [14]:
df.tail(3)

,A,B,C,D,E,F
1,1.0,2013-01-02,1.0,3,train,foo
2,1.0,2013-01-02,1.0,3,test,foo
3,1.0,2013-01-02,1.0,3,train,foo


혹은 list of row 와 같은 형식으로 입력할 수도 있습니다. 이때는 column name 을 따로 설정합니다.

In [15]:
data = [
    (29, 'mid',  'cloudy'),
    (34, 'high', 'sunny'),
    (36, 'high', 'rainy'),
    (32, 'mid',  'cloudy'),
    (30, 'low',  'sunny')
]
date = pd.date_range('20190601', periods=len(data), freq='D')
columns = 'temporature humidity weather'.split()

df = pd.DataFrame(data, columns=columns, index=date)
df

,temporature,humidity,weather
2019-06-01,29,mid,cloudy
2019-06-02,34,high,sunny
2019-06-03,36,high,rainy
2019-06-04,32,mid,cloudy
2019-06-05,30,low,sunny


Column name 과 index 는 이후에 수정가능합니다.

In [16]:
date = pd.date_range('20190701', periods=len(data), freq='D')
df.columns = 'temp hum class'.split()
df.index = date
df

,temp,hum,class
2019-07-01,29,mid,cloudy
2019-07-02,34,high,sunny
2019-07-03,36,high,rainy
2019-07-04,32,mid,cloudy
2019-07-05,30,low,sunny


## Select, slice

Column name 을 이용하면 해당 column 을 Series 의 형태로 선택할 수 있습니다.

In [17]:
print(type(df['temp']), end='\n\n')  # <class 'pandas.core.series.Series'>
df['temp']

<class 'pandas.core.series.Series'>



2019-07-01    29
2019-07-02    34
2019-07-03    36
2019-07-04    32
2019-07-05    30
Freq: D, Name: temp, dtype: int64

두 개 이상의 columns 을 선택하면 DataFrame 으로 선택됩니다.

In [18]:
print(type(df[['temp', 'class']]), end='\n\n')  # <class 'pandas.core.frame.DataFrame'>
df[['temp', 'class']]

<class 'pandas.core.frame.DataFrame'>



,temp,class
2019-07-01,29,cloudy
2019-07-02,34,sunny
2019-07-03,36,rainy
2019-07-04,32,cloudy
2019-07-05,30,sunny


이를 이용하면 column 의 순서를 재정렬 할 수 있습니다.

In [19]:
df[['class', 'temp', 'hum']]

,class,temp,hum
2019-07-01,cloudy,29,mid
2019-07-02,sunny,34,high
2019-07-03,rainy,36,high
2019-07-04,cloudy,32,mid
2019-07-05,sunny,30,low


Row 에 대해서도 파이썬의 리스트처럼 slice 를 할 수 있습니다.

In [20]:
df[1:3]

,temp,hum,class
2019-07-02,34,high,sunny
2019-07-03,36,high,rainy


그러나 하나의 row 를 선택하는 것은 하나의 column 을 선택하는 것과 혼동될 수 있기 때문에 다음처럼 slice 를 해야 합니다.

In [21]:
# df[0]
df[0:1]

,temp,hum,class
2019-07-01,29,mid,cloudy


하지만 위의 방법 역시 column slice 와 혼동됩니다. Row 를 number index 로 slicing 을 할 때에는 iloc 을 이용하는 것이 명확합니다.

In [22]:
df.iloc[0:2]

,temp,hum,class
2019-07-01,29,mid,cloudy
2019-07-02,34,high,sunny


혹은 특정 값의 index 를 이용하여 row 를 선택할 수도 있습니다. 이때는 loc 를 이용합니다.

In [23]:
df.loc[date[:2]]

,temp,hum,class
2019-07-01,29,mid,cloudy
2019-07-02,34,high,sunny


## Append / delete columns / rows

pandas 에서의 and 조건은 & 입니다.

In [24]:
df['play'] = (df['temp'] <= 30) & (df['class'] == 'sunny') 
df.head()

,temp,hum,class,play
2019-07-01,29,mid,cloudy,False
2019-07-02,34,high,sunny,False
2019-07-03,36,high,rainy,False
2019-07-04,32,mid,cloudy,False
2019-07-05,30,low,sunny,True


In [25]:
df

,temp,hum,class,play
2019-07-01,29,mid,cloudy,False
2019-07-02,34,high,sunny,False
2019-07-03,36,high,rainy,False
2019-07-04,32,mid,cloudy,False
2019-07-05,30,low,sunny,True


Column 이 제거된 새로운 DataFrame 이 return 됩니다. 이전 DataFrame 을 교체하려면 이를 변수로 지정하거나 `inplace=True` 로 설정해야 합니다.

In [26]:
# df.drop(['play'], axis=1)
df.drop(['play'], axis=1, inplace=True)
# df = df.drop(['play'], axis=1)
df.head()

,temp,hum,class
2019-07-01,29,mid,cloudy
2019-07-02,34,high,sunny
2019-07-03,36,high,rainy
2019-07-04,32,mid,cloudy
2019-07-05,30,low,sunny


새로운 DataFrame 을 하나 더 만듭니다.

In [27]:
df2 = pd.DataFrame(data,
    columns = 'temp hum class'.split(),
    index = pd.date_range('20190601', periods=len(data), freq='D'))
df2

,temp,hum,class
2019-06-01,29,mid,cloudy
2019-06-02,34,high,sunny
2019-06-03,36,high,rainy
2019-06-04,32,mid,cloudy
2019-06-05,30,low,sunny


두 DataFrame 을 더할 경우, 서로 다른 index 를 지녔기 때문에 병합이 일어나지 않습니다. 아래의 연산은 같은 index 를 지닌 부분끼리 그 값을 더하라는 의미입니다.

In [28]:
df2 + df

,temp,hum,class
2019-06-01,NaN,NaN,NaN
2019-06-02,NaN,NaN,NaN
2019-06-03,NaN,NaN,NaN
2019-06-04,NaN,NaN,NaN
2019-06-05,NaN,NaN,NaN
2019-07-01,NaN,NaN,NaN
2019-07-02,NaN,NaN,NaN
2019-07-03,NaN,NaN,NaN
2019-07-04,NaN,NaN,NaN
2019-07-05,NaN,NaN,NaN


Row 를 기준으로 두 개의 DataFrame 을 병합할 수 있습니다. 이때 column names 가 서로 다르면 병합이 되지 않습니다. `sort` 는 Row 가 아닌 column 에 대한 정렬입니다.

In [29]:
# df_m = pd.concat([df, df2], axis=0, sort=False)
df_m = pd.concat([df, df2], axis=0, sort=True)
df_m

,class,hum,temp
2019-07-01,cloudy,mid,29
2019-07-02,sunny,high,34
2019-07-03,rainy,high,36
2019-07-04,cloudy,mid,32
2019-07-05,sunny,low,30
2019-06-01,cloudy,mid,29
2019-06-02,sunny,high,34
2019-06-03,rainy,high,36
2019-06-04,cloudy,mid,32
2019-06-05,sunny,low,30


Index 기준으로 정렬된 새로운 DataFrame 이 return 됩니다. 이전 DataFrame 을 교체하려면 이를 변수로 지정해야 합니다.

In [30]:
# df_m.sort_index(axis=0, ascending=True)
df_m = df_m.sort_index(axis=0, ascending=True)
df_m

,class,hum,temp
2019-06-01,cloudy,mid,29
2019-06-02,sunny,high,34
2019-06-03,rainy,high,36
2019-06-04,cloudy,mid,32
2019-06-05,sunny,low,30
2019-07-01,cloudy,mid,29
2019-07-02,sunny,high,34
2019-07-03,rainy,high,36
2019-07-04,cloudy,mid,32
2019-07-05,sunny,low,30


## Merge (join)

Merge 는 두 DataFrame 의 공통된 column 을 기준으로 서로 다른 column 을 병합합니다. 데이터베이스에서의 join 과 같습니다.

In [31]:
A = pd.DataFrame({'color': ['green', 'yellow', 'red'], 'num':[1, 2, 3]})
A

,color,num
0,green,1
1,yellow,2
2,red,3


In [32]:
B = pd.DataFrame({'color': ['yellow', 'green', 'pink'], 'size':['S', 'M', 'L']})
B

,color,size
0,yellow,S
1,green,M
2,pink,L


Inner join 은 두 DataFrame 에 모두 등장하는 값만을 병합합니다.

In [33]:
#pd.merge(left=A, left=B, how='inner')
pd.merge(A, B, how='inner')

,color,num,size
0,green,1,M
1,yellow,2,S


Outer join 은 모든 값을 병합합니다.

In [34]:
pd.merge(A, B, how='outer')

,color,num,size
0,green,1.0,M
1,yellow,2.0,S
2,red,3.0,NaN
3,pink,NaN,L


Left join  은 left 의 DataFrame 에 등장한 row 를 기준으로 병합합니다.

In [35]:
pd.merge(A, B, how='left')

,color,num,size
0,green,1,M
1,yellow,2,S
2,red,3,NaN


Right join 은 right 의 DataFrame 에 등장한 row 를 기준으로 병합합니다.

In [36]:
pd.merge(A, B, how='right')

,color,num,size
0,green,1.0,M
1,yellow,2.0,S
2,pink,NaN,L


Join 후 NaN 은 `DataFrame.fillna` 함수를 이용하여 변경할 수 있습니다. 숫자, 문자 모두 입력 가능합니다.

In [37]:
#pd.merge(A, B, how='outer').fillna(0)
pd.merge(A, B, how='outer').fillna('')

,color,num,size
0,green,1,M
1,yellow,2,S
2,red,3,
3,pink,,L


병합 후 기존의 index 는 사라집니다.

In [38]:
B = pd.DataFrame(
    {'color': ['yellow', 'green', 'pink'], 'size':['S', 'M', 'L']},
    index = ['A', 'B', 'C']
)
B

,color,size
A,yellow,S
B,green,M
C,pink,L


In [39]:
pd.merge(A, B, how='outer')

,color,num,size
0,green,1.0,M
1,yellow,2.0,S
2,red,3.0,NaN
3,pink,NaN,L


A 의 `color` 에 'yellow' 를 두 개 준비합니다.

In [40]:
A = pd.DataFrame({'color': ['green', 'yellow', 'red', 'yellow'], 'num':[1, 2, 3, 4]})
A

,color,num
0,green,1
1,yellow,2
2,red,3
3,yellow,4


B 에도 `color` 에 'yellow' 를 두 개 준비합니다. index 도 'C' 를 중복해봅니다. index 가 중복되더라도 입력됩니다. Index 는 primary key 가 아닌, 값입니다.

In [41]:
B = pd.DataFrame(
    {'color': ['yellow', 'green', 'pink', 'yellow'], 'size':['S', 'M', 'L', 'XL']},
    index = ['A', 'B', 'C', 'C']
)
B

,color,size
A,yellow,S
B,green,M
C,pink,L
C,yellow,XL


값이 중복된 경우에는 모든 가능한 경우로 merge 를 진행합니다.

In [42]:
pd.merge(A, B, how='left')

,color,num,size
0,green,1,M
1,yellow,2,S
2,yellow,2,XL
3,red,3,NaN
4,yellow,4,S
5,yellow,4,XL


## Value Count

Seaborn 의 iris dataset 을 이용합니다.

In [43]:
import seaborn as sns
iris = sns.load_dataset("iris")
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


Categorical variable 의 경우 `value_counts` 함수를 이용하면 각 값의 개수를 확인할 수 있습니다.

In [44]:
iris['species'].value_counts()

versicolor    50
virginica     50
setosa        50
Name: species, dtype: int64

Numerical variable 의 경우 `value_counts` 함수를 이용하면 각 값마다 빈도수를 계산합니다. 연속형 변수의 경우 주로 그 목적은 histrogram 이므로 `bins` 의 개수를 설정합니다.

In [45]:
# iris['sepal_width'].value_counts() # 모든 값에 대한 count
# 빈도수 기준 정렬
iris['sepal_width'].value_counts(bins=5)

(2.96, 3.44]     68
(2.48, 2.96]     46
(3.44, 3.92]     21
(1.997, 2.48]    11
(3.92, 4.4]       4
Name: sepal_width, dtype: int64

그러나 count 기준 정렬이 기본이기 때문에 range 기준 정렬을 위하여 정렬을 수행합니다.

In [46]:
# Bin range 기준 정렬
iris['sepal_width'].value_counts(bins=5).sort_index(axis=0)

(1.997, 2.48]    11
(2.48, 2.96]     46
(2.96, 3.44]     68
(3.44, 3.92]     21
(3.92, 4.4]       4
Name: sepal_width, dtype: int64

## Group by

group by 는 특정 column 기준으로 데이터를 병합합니다.

In [47]:
iris.groupby(['species']) # <pandas.core.groupby.generic.DataFrameGroupBy object at 0x7f3810cc74a8>
iris.groupby(['species']).mean()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


Iris 의 종별로 sepal width 의 historgram 을 그릴 수도 있습니다.

In [48]:
histogram_by_species = iris.groupby(['species'])['sepal_width'].value_counts(bins=5).sort_index(axis=0)
print(type(histogram_by_species), end='\n\n')
histogram_by_species

<class 'pandas.core.series.Series'>



species                               
setosa      (2.297, 2.72]                  1
            (2.72, 3.14]                  11
            (3.14, 3.56]                  22
            (3.56, 3.98]                  12
            (3.98, 4.4]                    4
versicolor  (1.9980000000000002, 2.28]     3
            (2.28, 2.56]                  10
            (2.56, 2.84]                  14
            (2.84, 3.12]                  18
            (3.12, 3.4]                    5
virginica   (2.197, 2.52]                  5
            (2.52, 2.84]                  14
            (2.84, 3.16]                  18
            (3.16, 3.48]                  10
            (3.48, 3.8]                    3
Name: sepal_width, dtype: int64

Series 를 row 로 이용하여 이를 DataFrame 으로 바꿀 수 있습니다. 하지만 groupby 와 value_count 를 이용했기 때문에 two columns (nested) index 가 만들어집니다.

In [49]:
sepal_width_by_species = pd.DataFrame(
    {'count': histogram_by_species}
)
sepal_width_by_species.head(10)

count
species                                     
setosa     (2.297, 2.72]                   1
           (2.72, 3.14]                   11
           (3.14, 3.56]                   22
           (3.56, 3.98]                   12
           (3.98, 4.4]                     4
versicolor (1.9980000000000002, 2.28]      3
           (2.28, 2.56]                   10
           (2.56, 2.84]                   14
           (2.84, 3.12]                   18
           (3.12, 3.4]                     5

`reset_index()` 를 이용하면 index 가 각각 column 으로 분해됩니다. binning 기준은 본래 존재하지 않았던 변수이기 때문에 level_1 이라는 임의의 이름이 부여되었습니다. `rename()` 함수를 이용하여 이를 변환할 수 있습니다.

In [50]:
sepal_width_by_species = pd.DataFrame({'count': histogram_by_species})
sepal_width_by_species = sepal_width_by_species.reset_index()
# sepal_width_by_species.rename(columns = {'level_1' : 'range'}, inplace = True)
sepal_width_by_species.head()

,species,level_1,count
0,setosa,"(2.297, 2.72]",1
1,setosa,"(2.72, 3.14]",11
2,setosa,"(3.14, 3.56]",22
3,setosa,"(3.56, 3.98]",12
4,setosa,"(3.98, 4.4]",4


'setosa' 종에 대한 histrogram 만을 선택해봅니다.

In [51]:
histogram_by_species['setosa']

(2.297, 2.72]     1
(2.72, 3.14]     11
(3.14, 3.56]     22
(3.56, 3.98]     12
(3.98, 4.4]       4
Name: sepal_width, dtype: int64

이 값은 interval 과 value 로 이뤄져 있습니다. 이들의 값을 정리하면 이후 histogram 의 그림에도 이용할 수 있습니다.

In [52]:
interval = list(histogram_by_species['setosa'].index)[0]
print(interval)
print(type(interval))
print(interval.left, interval.right)
print(list(histogram_by_species['setosa']))

(2.297, 2.72]
<class 'pandas._libs.interval.Interval'>
2.297 2.72
[1, 11, 22, 12, 4]


Bokeh 를 이용하여 histrogram 을 그리는 코드입니다. Bokeh 에 대한 설명은 bokeh tutorial 을 참고하세요.

In [53]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource

output_notebook()

intervals = list(histogram_by_species['setosa'].index)
values = list(histogram_by_species['setosa'])
width = intervals[0].right - intervals[0].left

data = {
    'position': [(interval.left + interval.right)/2 for interval in intervals],
    'top': values
}
source = ColumnDataSource(data)

p = figure(width=400, height=400)
p.vbar(x='position', width=width, top='top', source=source, alpha=0.5)
show(p)

Loading BokehJS ...

## Read data

CSV 형식의 파일은 `read_csv` 를 통하여 읽어올 수 있습니다.

In [54]:
pima_indians_data_paths = [
    '../mydata/data/datafile/pima-indians-diabetes.data',
    '../mydata/data/datafile/pima-indians-diabetes.names'
]

data = pd.read_csv(pima_indians_data_paths[0], delimiter=',')
data.head()

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


첫 줄을 column names 으로 인식합니다. header 가 없는 경우에는 이를 `None` 으로 설정합니다. header 는 columns 로 따로 지정하면 됩니다.

In [55]:
with open(pima_indians_data_paths[1], encoding='utf-8') as f:
    columns = [line.strip() for line in f]

data = pd.read_csv(pima_indians_data_paths[0], header=None)
data.columns = columns
data.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


`describe` 함수는 간단한 통계를 계산합니다.

In [56]:
data.describe()

,preg,plas,pres,skin,test,mass,pedi,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


파일 주소는 로컬 파일 뿐 아니라 URL 도 지원됩니다.

In [57]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
data = pd.read_csv(url, header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


## Readings

- https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html#
- https://thispointer.com/data-analysis-in-python-using-pandas/